In [3]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 50)

import os
# Use this to see how much memory the dataframes use
from sys import getsizeof

import yaml
import re

import pseudopeople as psp
from pseudopeople.configuration import get_configuration
from vivarium.framework.randomness import RandomnessStream
from vivarium.config_tree import ConfigTree

from vivarium_research_prl.utils import sizemb, MappingViaAttributes
from vivarium_research_prl import datatypes, alpha, data_loading
from vivarium_research_prl.noise import corruption, fake_names, noisify
from vivarium_research_prl.find_kids import datasets, noisify_data


!date
!whoami
!uname -a
!pwd

Tue 02 May 2023 06:05:50 PM PDT
ndbs
Linux int-slurm-sarchive-p0002 5.4.0-144-generic #161-Ubuntu SMP Fri Feb 3 14:49:04 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/pseudopeople_testing


In [4]:
%load_ext autoreload
%autoreload 2

# Data!

U.S.
```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v2.1_updated_emigration/united_states_of_america/2023_04_19_16_30_28/final_results/2023_04_20_11_23_26/pseudopeople_input_data_usa_1.0.0
```

Rhode Island
```
/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/v2.1_updated_emigration/united_states_of_america/2023_04_19_16_30_28/final_results/2023_04_20_11_23_26/states/pseudopeople_input_data_ri_1.0.0
```

In [5]:
project_dir = '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
model_dir = (
    f'{project_dir}/results'
    '/v2.1_updated_emigration/united_states_of_america/2023_04_19_16_30_28'
)
run_dir = f'{model_dir}/final_results/2023_04_20_11_23_26'
rhode_island_dir = f'{run_dir}/states/pseudopeople_input_data_ri_1.0.0'
usa_dir = f'{run_dir}/pseudopeople_input_data_usa_1.0.0'

!ls -halt $rhode_island_dir

total 260K
drwxrwsr-x 3 rmudambi IHME-Simulationscience 1.0K Apr 21 16:01 ..
drwxrwsr-x 2 rmudambi IHME-Simulationscience 167K Apr 21 14:55 current_population_survey
drwxrwsr-x 2 rmudambi IHME-Simulationscience 167K Apr 21 14:54 taxes_dependents
drwxrwsr-x 2 rmudambi IHME-Simulationscience 167K Apr 21 14:53 women_infants_and_children
drwxrwsr-x 2 rmudambi IHME-Simulationscience 167K Apr 21 14:53 american_community_survey
drwxrwsr-x 2 rmudambi IHME-Simulationscience 167K Apr 21 14:53 decennial_census
drwxrwsr-x 2 rmudambi IHME-Simulationscience 167K Apr 21 14:52 taxes_1040
drwxrwsr-x 2 rmudambi IHME-Simulationscience 167K Apr 21 14:50 taxes_w2_and_1099
drwxrwsr-x 9 rmudambi IHME-Simulationscience 3.5K Apr 20 16:02 .


# Test configuration interface

In [13]:
config = psp.get_config()
config.keys()

dict_keys(['decennial_census', 'american_community_survey', 'current_population_survey', 'women_infants_and_children', 'social_security', 'taxes_w2_and_1099'])

In [8]:
config['decennial_census']

{'row_noise': {'omit_row': {'row_probability': 0.0145}},
 'column_noise': {'first_name': {'leave_blank': {'cell_probability': 0.01},
   'use_fake_name': {'cell_probability': 0.01},
   'make_typos': {'cell_probability': 0.01, 'token_probability': 0.1}},
  'middle_initial': {'leave_blank': {'cell_probability': 0.01},
   'make_typos': {'cell_probability': 0.01, 'token_probability': 0.1}},
  'last_name': {'leave_blank': {'cell_probability': 0.01},
   'use_fake_name': {'cell_probability': 0.01},
   'make_typos': {'cell_probability': 0.01, 'token_probability': 0.1}},
  'age': {'leave_blank': {'cell_probability': 0.01},
   'misreport_age': {'cell_probability': 0.01,
    'possible_age_differences': {-2: 0.1, -1: 0.4, 1: 0.4, 2: 0.1}},
   'make_typos': {'cell_probability': 0.01, 'token_probability': 0.1}},
  'date_of_birth': {'leave_blank': {'cell_probability': 0.01},
   'write_wrong_digits': {'cell_probability': 0.01, 'token_probability': 0.1},
   'make_typos': {'cell_probability': 0.01, 'toke

In [9]:
census_config = psp.get_config('decennial_census')
census_config

{'row_noise': {'omit_row': {'row_probability': 0.0145}},
 'column_noise': {'first_name': {'leave_blank': {'cell_probability': 0.01},
   'use_fake_name': {'cell_probability': 0.01},
   'make_typos': {'cell_probability': 0.01, 'token_probability': 0.1}},
  'middle_initial': {'leave_blank': {'cell_probability': 0.01},
   'make_typos': {'cell_probability': 0.01, 'token_probability': 0.1}},
  'last_name': {'leave_blank': {'cell_probability': 0.01},
   'use_fake_name': {'cell_probability': 0.01},
   'make_typos': {'cell_probability': 0.01, 'token_probability': 0.1}},
  'age': {'leave_blank': {'cell_probability': 0.01},
   'misreport_age': {'cell_probability': 0.01,
    'possible_age_differences': {-2: 0.1, -1: 0.4, 1: 0.4, 2: 0.1}},
   'make_typos': {'cell_probability': 0.01, 'token_probability': 0.1}},
  'date_of_birth': {'leave_blank': {'cell_probability': 0.01},
   'write_wrong_digits': {'cell_probability': 0.01, 'token_probability': 0.1},
   'make_typos': {'cell_probability': 0.01, 'toke

# Write a function to get a config with zero noise

In [10]:
def get_zero_noise_config(row_or_col='both'):
    if row_or_col not in ['row', 'column', 'both']:
        raise ValueError("row_or_col must be 'row', 'column', or 'both'")
    config = psp.get_config()
    for dataset_config in config.values():
        if row_or_col in ['row', 'both']:
            for row_noise_config in dataset_config['row_noise'].values():
                row_noise_config['row_probability'] = 0
        if row_or_col in ['column', 'both']:
            for column_config in dataset_config['column_noise'].values():
                for noise_config in column_config.values():
                    if 'row_probability' in noise_config:
                        noise_config['row_probability'] = 0
                    elif 'cell_probability' in noise_config:
                        noise_config['cell_probability'] = 0
    return config

zero_config = get_zero_noise_config()
zero_config['decennial_census']

{'row_noise': {'omit_row': {'row_probability': 0}},
 'column_noise': {'first_name': {'leave_blank': {'cell_probability': 0},
   'use_fake_name': {'cell_probability': 0},
   'make_typos': {'cell_probability': 0, 'token_probability': 0.1}},
  'middle_initial': {'leave_blank': {'cell_probability': 0},
   'make_typos': {'cell_probability': 0, 'token_probability': 0.1}},
  'last_name': {'leave_blank': {'cell_probability': 0},
   'use_fake_name': {'cell_probability': 0},
   'make_typos': {'cell_probability': 0, 'token_probability': 0.1}},
  'age': {'leave_blank': {'cell_probability': 0},
   'misreport_age': {'cell_probability': 0,
    'possible_age_differences': {-2: 0.1, -1: 0.4, 1: 0.4, 2: 0.1}},
   'make_typos': {'cell_probability': 0, 'token_probability': 0.1}},
  'date_of_birth': {'leave_blank': {'cell_probability': 0},
   'write_wrong_digits': {'cell_probability': 0, 'token_probability': 0.1},
   'make_typos': {'cell_probability': 0, 'token_probability': 0.1}},
  'street_number': {'lea

In [12]:
# Abie's solution
def recursive_zero(d):
    if not isinstance(d, dict):
        return

    for k in d.keys():
        if 'probability' in str(k):
            d[k] = 0.0
        else:
            recursive_zero(d[k])
            
zero_config2 = psp.get_config()
recursive_zero(zero_config2)
zero_config2['decennial_census']

{'row_noise': {'omit_row': {'row_probability': 0.0}},
 'column_noise': {'first_name': {'leave_blank': {'cell_probability': 0.0},
   'use_fake_name': {'cell_probability': 0.0},
   'make_typos': {'cell_probability': 0.0, 'token_probability': 0.0}},
  'middle_initial': {'leave_blank': {'cell_probability': 0.0},
   'make_typos': {'cell_probability': 0.0, 'token_probability': 0.0}},
  'last_name': {'leave_blank': {'cell_probability': 0.0},
   'use_fake_name': {'cell_probability': 0.0},
   'make_typos': {'cell_probability': 0.0, 'token_probability': 0.0}},
  'age': {'leave_blank': {'cell_probability': 0.0},
   'misreport_age': {'cell_probability': 0.0,
    'possible_age_differences': {-2: 0.1, -1: 0.4, 1: 0.4, 2: 0.1}},
   'make_typos': {'cell_probability': 0.0, 'token_probability': 0.0}},
  'date_of_birth': {'leave_blank': {'cell_probability': 0.0},
   'write_wrong_digits': {'cell_probability': 0.0, 'token_probability': 0.0},
   'make_typos': {'cell_probability': 0.0, 'token_probability': 0

# Generate WIC data for Rhode Island

In [18]:
%%time
wic = psp.generate_women_infants_and_children(rhode_island_dir)
wic

Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Noising data:  46%|██████████████████████▎                         | 155/334 [00:55<01:04,  2.76it/s]


Noising data:  58%|███████████████████████████▉                    | 194/334 [01:10<00:51,  2.74it/s]


...

Noising data:  81%|███████████████████████████████████████         | 272/334 [01:39<00:23,  2.68it/s]


Noising data:  93%|████████████████████████████████████████████▋   | 311/334 [01:53<00:08,  2.69it/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]
                                                                                                     

CPU times: user 1min 54s, sys: 6.3 s, total: 2min
Wall time: 2min 2s


,household_id,simulant_id,first_name,middle_initial,last_name,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,sex,race_ethnicity
0,2599_16688,2599_41986,Sarah,C,Mccarty,06/02/1983,2498,e interstate 30,NaN,cranston,RI,02840,Female,White
1,2599_17590,2599_44131,Nicole,A,Lewis,05/22/1991,NaN,9th avnu,NaN,west warwick,RI,02888,Female,White
2,2599_68676,2599_171233,Lindsay,J,Milligan,07/27/1988,2121,winter rdg ln,apt 101,west warwick,RI,02861,Female,White
3,2599_75319,2599_187606,Jennifer,C,Granados Rodriguez,12/21/1984,9824,holly ave,NaN,cranston,RI,02860,Female,Latino
4,2599_75596,2599_188275,Kerry,J,Noel,07/10/1996,2124,justbrand lane,NaN,newport,RI,02920,Female,Black
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14425,7745_214563,7745_1010123,Liam,J,Newman,08/08/2019,46,haverford dr,NaN,south kingstown,RI,02813,Male,White
14426,7745_31844,7745_1012353,Isaac,I,Dellinger,09/21/2019,e1853,starview ln,NaN,e greenwich,RI,02831,Male,White
14427,7745_228895,7745_1014120,Elizabeth,N,Coker,10/24/2019,807,road 56,NaN,east providence,RI,02864,Female,Asian
14428,7745_61219,7745_1016457,Isabel,I,Perez-Flores,12/30/2019,17419,park dr s,NaN,middletown,RI,02915,Female,Latino


In [25]:
sizemb(wic)

9.811167

# Generate WIC data with no noise

## And try passing a sub-configuration to see if it works

### Nope, doesn't work. I think we should fix this.

In [15]:
wic0_config = psp.get_config('women_infants_and_children')
recursive_zero(wic0_config)
wic0_config

{'row_noise': {'omit_row': {'row_probability': 0.0}},
 'column_noise': {'first_name': {'leave_blank': {'cell_probability': 0.0},
   'use_fake_name': {'cell_probability': 0.0},
   'make_typos': {'cell_probability': 0.0, 'token_probability': 0.0}},
  'middle_initial': {'leave_blank': {'cell_probability': 0.0},
   'make_typos': {'cell_probability': 0.0, 'token_probability': 0.0}},
  'last_name': {'leave_blank': {'cell_probability': 0.0},
   'use_fake_name': {'cell_probability': 0.0},
   'make_typos': {'cell_probability': 0.0, 'token_probability': 0.0}},
  'date_of_birth': {'leave_blank': {'cell_probability': 0.0},
   'write_wrong_digits': {'cell_probability': 0.0, 'token_probability': 0.0},
   'make_typos': {'cell_probability': 0.0, 'token_probability': 0.0}},
  'street_number': {'leave_blank': {'cell_probability': 0.0},
   'write_wrong_digits': {'cell_probability': 0.0, 'token_probability': 0.0},
   'make_typos': {'cell_probability': 0.0, 'token_probability': 0.0}},
  'street_name': {'le

In [20]:
%%time
wic0 = psp.generate_women_infants_and_children(rhode_island_dir, config=wic0_config)
wic0

ConfigurationError: Invalid dataset 'row_noise' provided. Valid datasets are ['decennial_census', 'american_community_survey', 'current_population_survey', 'women_infants_and_children', 'social_security', 'taxes_w2_and_1099'].

In [21]:
%%time
wic0 = psp.generate_women_infants_and_children(rhode_island_dir, config=zero_config)
wic0

Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


...

Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]
                                                                                                     

CPU times: user 1min 47s, sys: 5.32 s, total: 1min 52s
Wall time: 1min 52s


,household_id,simulant_id,first_name,middle_initial,last_name,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,sex,race_ethnicity
0,2599_16688,2599_41986,Sarah,C,Mccarty,06/02/1983,2498,e interstate 30,NaN,cranston,RI,02840,Female,White
1,2599_17590,2599_44131,Nicole,A,Lewis,05/22/1991,NaN,9th avnu,NaN,west warwick,RI,02888,Female,White
2,2599_68676,2599_171233,Lindsay,J,Milligan,07/27/1988,2121,winter rdg ln,apt 101,west warwick,RI,02861,Female,White
3,2599_75319,2599_187606,Jennifer,C,Granados Rodriguez,12/21/1984,9824,holly ave,NaN,cranston,RI,02860,Female,Latino
4,2599_75596,2599_188275,Kerry,J,Noel,07/10/1996,2124,justbrand lane,NaN,newport,RI,02920,Female,Black
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14425,7745_214563,7745_1010123,Liam,J,Newman,08/08/2019,46,haverford dr,NaN,south kingstown,RI,02813,Male,White
14426,7745_31844,7745_1012353,Isaac,I,Dellinger,09/21/2019,e1853,starview ln,NaN,e greenwich,RI,02831,Male,White
14427,7745_228895,7745_1014120,Elizabeth,N,Coker,10/24/2019,807,road 56,NaN,east providence,RI,02864,Female,Asian
14428,7745_61219,7745_1016457,Isabel,I,Perez-Flores,12/30/2019,17419,park dr s,NaN,middletown,RI,02915,Female,Latino


# Check datatypes

In [22]:
wic.dtypes

household_id        object
simulant_id         object
first_name          object
middle_initial      object
last_name           object
date_of_birth       object
street_number       object
street_name         object
unit_number         object
city                object
state             category
zipcode             object
sex               category
race_ethnicity    category
dtype: object

In [24]:
# Good, looks like everything is either string or NaN
wic.apply(lambda col: col.map(type).unique())

household_id                       [<class 'str'>]
simulant_id                        [<class 'str'>]
first_name        [<class 'str'>, <class 'float'>]
middle_initial    [<class 'str'>, <class 'float'>]
last_name         [<class 'str'>, <class 'float'>]
date_of_birth                      [<class 'str'>]
street_number     [<class 'str'>, <class 'float'>]
street_name       [<class 'str'>, <class 'float'>]
unit_number       [<class 'float'>, <class 'str'>]
city                               [<class 'str'>]
state                         [<class 'str'>, nan]
zipcode                            [<class 'str'>]
sex                           [<class 'str'>, nan]
race_ethnicity                     [<class 'str'>]
dtype: object

In [26]:
# Good, looks like there are no more empty strings for missing data
(wic == '').sum()

household_id      0
simulant_id       0
first_name        0
middle_initial    0
last_name         0
date_of_birth     0
street_number     0
street_name       0
unit_number       0
city              0
state             0
zipcode           0
sex               0
race_ethnicity    0
dtype: int64

# Generate census data

In [27]:
%%time
census = psp.generate_decennial_census(rhode_island_dir)
census

Applying noise: 100%|████████████████████████████████████████████████| 8/8 [00:00<00:00, 31.46type/s]


Applying noise: 100%|████████████████████████████████████████████████| 8/8 [00:00<00:00, 31.51type/s]


Applying noise: 100%|████████████████████████████████████████████████| 8/8 [00:00<00:00, 34.68type/s]


Applying noise: 100%|████████████████████████████████████████████████| 8/8 [00:00<00:00, 31.67type/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


...

Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Applying noise: 100%|████████████████████████████████████████████████| 8/8 [00:00<00:00, 29.69type/s]


CPU times: user 9min 45s, sys: 1min 36s, total: 11min 22s
Wall time: 11min 13s


,simulant_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,relation_to_reference_person,sex,race_ethnicity
0,2477_591,John,T,Everetts,82,03/22/1938,6292,e secretariat dr,NaN,providence,RI,02886,Reference person,Male,White
1,2477_2188,Susan,F,Dorn,63,01/08/1957,41468,e chenango pl,NaN,jamestown,RI,02886,Reference person,Female,White
2,2477_2189,John,P,Dorn,75,02/07/1945,41468,e chenango pl,NaN,jamestown,RI,02886,Opp-sex spouse,Male,White
3,2477_2465,Kassandra,R,NaN,26,06/79/1993,138,elastic plant rd,NaN,cranston,RI,02814,Reference person,Female,Latino
4,2477_2466,Sophia,E,Medrano Garcia,7,05/07/2012,130,elastic plant rd,NaN,cranston,RI,02814,Biological child,Female,Latino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072363,8133_1021295,Arielle,T,Parrella,17,03/26/2003,106,parkview ave,NaN,pawtucket,RI,02888,Biological child,Female,Latino
1072364,8133_1021308,Devin,B,Scoggins,9,11/18/2010,5758,c.r. 2461,NaN,warwick,RI,02891,Biological child,Male,White
1072365,8133_1021349,Braden,A,Corbett,22,05/10/1997,68,harbor woods cir,NaN,lincoln,RI,02903,Roommate,Male,Black
1072366,8133_1021967,Cameron,A,Horan,0,04/09/2020,7120,w franklin ave,NaN,johnston,RI,02909,Other nonrelative,Male,White


In [28]:
sizemb(census)

695.035011

In [29]:
census.dtypes

simulant_id                       object
first_name                        object
middle_initial                    object
last_name                         object
age                               object
date_of_birth                     object
street_number                     object
street_name                       object
unit_number                       object
city                              object
state                           category
zipcode                           object
relation_to_reference_person    category
sex                             category
race_ethnicity                  category
dtype: object

In [30]:
%%time
census_c = census.copy()
datatypes.convert_to_int_and_categorical(census_c)
census_c.dtypes

CPU times: user 6.98 s, sys: 400 ms, total: 7.38 s
Wall time: 7.38 s


simulant_id                        int64
first_name                      category
middle_initial                  category
last_name                       category
age                             category
date_of_birth                   category
street_number                   category
street_name                     category
unit_number                     category
city                            category
state                           category
zipcode                         category
relation_to_reference_person    category
sex                             category
race_ethnicity                  category
dtype: object

In [31]:
sizemb(census_c)

90.590814

In [32]:
census_c

,simulant_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,relation_to_reference_person,sex,race_ethnicity
0,2477000000591,John,T,Everetts,82,03/22/1938,6292,e secretariat dr,NaN,providence,RI,02886,Reference person,Male,White
1,2477000002188,Susan,F,Dorn,63,01/08/1957,41468,e chenango pl,NaN,jamestown,RI,02886,Reference person,Female,White
2,2477000002189,John,P,Dorn,75,02/07/1945,41468,e chenango pl,NaN,jamestown,RI,02886,Opp-sex spouse,Male,White
3,2477000002465,Kassandra,R,NaN,26,06/79/1993,138,elastic plant rd,NaN,cranston,RI,02814,Reference person,Female,Latino
4,2477000002466,Sophia,E,Medrano Garcia,7,05/07/2012,130,elastic plant rd,NaN,cranston,RI,02814,Biological child,Female,Latino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072363,8133001021295,Arielle,T,Parrella,17,03/26/2003,106,parkview ave,NaN,pawtucket,RI,02888,Biological child,Female,Latino
1072364,8133001021308,Devin,B,Scoggins,9,11/18/2010,5758,c.r. 2461,NaN,warwick,RI,02891,Biological child,Male,White
1072365,8133001021349,Braden,A,Corbett,22,05/10/1997,68,harbor woods cir,NaN,lincoln,RI,02903,Roommate,Male,Black
1072366,8133001021967,Cameron,A,Horan,0,04/09/2020,7120,w franklin ave,NaN,johnston,RI,02909,Other nonrelative,Male,White


# Generate W2/1099 data

In [33]:
%%time
w2 = psp.generate_taxes_w2_and_1099(rhode_island_dir)
w2

Applying noise:  75%|████████████████████████████████████            | 6/8 [00:00<00:00, 46.83type/s]


Applying noise:  75%|████████████████████████████████████            | 6/8 [00:00<00:00, 48.34type/s]


Applying noise:  75%|████████████████████████████████████            | 6/8 [00:00<00:00, 50.73type/s]


Applying noise:  75%|████████████████████████████████████            | 6/8 [00:00<00:00, 44.35type/s]


Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


...

Applying noise:   0%|                                                        | 0/8 [00:00<?, ?type/s]


Noising data:  93%|████████████████████████████████████████████▌   | 310/334 [26:36<02:06,  5.28s/it]


Applying noise:  75%|████████████████████████████████████            | 6/8 [00:00<00:00, 52.66type/s]


CPU times: user 25min 20s, sys: 3min 54s, total: 29min 15s
Wall time: 28min 44s


,simulant_id,first_name,middle_initial,last_name,age,date_of_birth,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,...,income,employer_id,employer_name,employer_street_number,employer_street_name,employer_unit_number,employer_city,employer_state,employer_zipcode,tax_form
0,1490_588,Adriana,M,Baumert,41,02/25/1979,1029,westgate way,NaN,NaN,...,55820,325622,Westminster Arts,NaN,e federal st,NaN,birmingham,AL,35648,W2
1,1490_589,David,L,Baumert,39,03/10/1981,1029,westgate way,NaN,NaN,...,77760,1598657,Empanadas Golden Boys Painting,6628,hawks haven rd,NaN,statesville,NC,28168,W2
2,1490_590,Abigael,M,Baumert,22,05/07/1998,1029,westgate way,NaN,NaN,...,9451,849149,Alamo Community Church of Education,9712,3rd street,NaN,five forks,SC,29926,W2
3,1490_1161,Ronald,D,Wright,38,02/23/1982,2115,nort elk lane,NaN,NaN,...,12349,99503,Northwest Pizza & Boutique,3223,wellesley avenue,NaN,winter springs,FL,32571,W2
4,1490_1161,Ronald,NaN,Wright,38,02/23/1982,2115,nort elk lane,NaN,NaN,...,7196,217787,Saint Helens Nail Spa,155,7th avenue,NaN,new york,NY,11217,W2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043909,5072_1025338,Joshua,NaN,Beil,30,07/20/1990,1425,old kings hwy s,NaN,NaN,...,16370,413828,Henry E DDS,3,nantucket dr,NaN,beckley,WV,25701,W2
1043910,5072_1029528,Marcus,D,Salu Jacquot,26,11/21/1994,422,katherine claire ln,NaN,NaN,...,8159,568939,GCS University Settlement Early Childhood Educ...,833,w pike st,NaN,flint,MI,48122,W2
1043911,5072_1030814,Amelia,T,Alcamo,65,07/08/1955,408,lake ridge drive nrth,NaN,NaN,...,4964,771438,The Church of Touch Miami,722,everbreeze dr,NaN,fort myers,FL,32826,W2
1043912,5072_1030814,Amelia,T,Alcamo,65,07/08/1955,408,lake ridge drive nrth,NaN,NaN,...,10930,1203472,Green Gables Estate Bellevue Dental,10005,n crescent dr,NaN,paradise,TX,75020,W2


In [34]:
sizemb(w2)

1243.378012

In [35]:
w2.dtypes

simulant_id                        object
first_name                         object
middle_initial                     object
last_name                          object
age                                object
date_of_birth                      object
mailing_address_street_number      object
mailing_address_street_name        object
mailing_address_unit_number        object
mailing_address_po_box             object
mailing_address_city               object
mailing_address_state            category
mailing_address_zipcode            object
ssn                                object
income                             object
employer_id                        object
employer_name                      object
employer_street_number             object
employer_street_name               object
employer_unit_number               object
employer_city                      object
employer_state                   category
employer_zipcode                   object
tax_form                         c

In [37]:
%%time
w2_c = w2.copy()
datatypes.convert_to_int_and_categorical(w2_c)
w2_c.dtypes

CPU times: user 18.1 s, sys: 1.13 s, total: 19.3 s
Wall time: 19.3 s


simulant_id                         int64
first_name                       category
middle_initial                   category
last_name                        category
age                              category
date_of_birth                    category
mailing_address_street_number    category
mailing_address_street_name      category
mailing_address_unit_number      category
mailing_address_po_box           category
mailing_address_city             category
mailing_address_state            category
mailing_address_zipcode          category
ssn                                 int32
income                           category
employer_id                      category
employer_name                    category
employer_street_number           category
employer_street_name             category
employer_unit_number             category
employer_city                    category
employer_state                   category
employer_zipcode                 category
tax_form                         c

In [38]:
sizemb(w2_c)

277.230335

In [58]:
%%time
100 * w2_c.apply(sizemb) / w2.apply(sizemb)

CPU times: user 2.89 s, sys: 0 ns, total: 2.89 s
Wall time: 2.88 s


simulant_id                       11.800439
first_name                         4.749150
middle_initial                     1.736546
last_name                         19.216926
age                                5.569373
date_of_birth                     10.903457
mailing_address_street_number     12.695500
mailing_address_street_name       41.349076
mailing_address_unit_number        8.836174
mailing_address_po_box             8.175968
mailing_address_city               3.576431
mailing_address_state            100.000000
mailing_address_zipcode            3.404489
ssn                                5.919064
income                            40.392641
employer_id                      116.813893
employer_name                     82.212841
employer_street_number            13.024058
employer_street_name              47.003967
employer_unit_number              10.472774
employer_city                      5.988089
employer_state                   100.000000
employer_zipcode                

In [41]:
w2_c.drop(columns=['simulant_id', 'ssn']).apply(lambda col: 100*len(col.cat.categories)/len(col))

first_name                        1.140994
middle_initial                    0.002491
last_name                         9.406522
age                               0.018392
date_of_birth                     3.342038
mailing_address_street_number     4.019776
mailing_address_street_name      23.161582
mailing_address_unit_number       1.207571
mailing_address_po_box            1.075089
mailing_address_city              0.365739
mailing_address_state             0.004885
mailing_address_zipcode           0.112940
income                           17.229868
employer_id                      60.810756
employer_name                    56.693655
employer_street_number            4.328709
employer_street_name             28.889066
employer_unit_number              1.533555
employer_city                     2.122014
employer_state                    0.004885
employer_zipcode                  3.159264
tax_form                          0.000192
dtype: float64

In [42]:
census_c.drop(columns=['simulant_id']).apply(lambda col: 100*len(col.cat.categories)/len(col))

first_name                       1.453139
middle_initial                   0.002425
last_name                        9.667390
age                              0.021168
date_of_birth                    4.167692
street_number                    3.889243
street_name                     24.382861
unit_number                      1.293866
city                             0.275092
state                            0.004756
zipcode                          0.126822
relation_to_reference_person     0.001772
sex                              0.000187
race_ethnicity                   0.000653
dtype: float64

In [43]:
w2.employer_id

0           325622
1          1598657
2           849149
3            99503
4           217787
            ...   
1043909     413828
1043910     568939
1043911     771438
1043912    1203472
1043913    1124899
Name: employer_id, Length: 1043914, dtype: object

In [44]:
w2_c.employer_id

0           325622
1          1598657
2           849149
3            99503
4           217787
            ...   
1043909     413828
1043910     568939
1043911     771438
1043912    1203472
1043913    1124899
Name: employer_id, Length: 1043914, dtype: category
Categories (634812, object): [1, 2, 3, 6, ..., '999615 ', '999833', '999922', '999964 ']

In [45]:
w2.apply(lambda col: col.map(type).unique())

simulant_id                                                      [<class 'str'>]
first_name                                      [<class 'str'>, <class 'float'>]
middle_initial                                  [<class 'str'>, <class 'float'>]
last_name                                       [<class 'str'>, <class 'float'>]
age                              [<class 'int'>, <class 'str'>, <class 'float'>]
date_of_birth                                   [<class 'str'>, <class 'float'>]
mailing_address_street_number                   [<class 'str'>, <class 'float'>]
mailing_address_street_name                     [<class 'str'>, <class 'float'>]
mailing_address_unit_number                     [<class 'float'>, <class 'str'>]
mailing_address_po_box                          [<class 'float'>, <class 'str'>]
mailing_address_city                            [<class 'str'>, <class 'float'>]
mailing_address_state                                       [<class 'str'>, nan]
mailing_address_zipcode     

In [47]:
%time w2.income.map(type).value_counts()

CPU times: user 100 ms, sys: 3.41 ms, total: 104 ms
Wall time: 101 ms


<class 'int'>      1012641
<class 'str'>        20549
<class 'float'>      10724
Name: income, dtype: int64

In [49]:
%time w2_c.income.map(type).value_counts()

CPU times: user 109 ms, sys: 3.54 ms, total: 113 ms
Wall time: 110 ms


<class 'int'>    1012641
<class 'str'>      20549
Name: income, dtype: int64

In [50]:
w2.income.loc[w2.income.map(type) == str]

13           10393
33            5823
63           64044
75           33047
220          85849
            ...   
1043641     19775 
1043654      62139
1043789    1511776
1043811      22784
1043819     306332
Name: income, Length: 20549, dtype: object

In [54]:
w2_c.income.loc[w2.income.notna() & ~w2_c.income.astype(str).str.isdigit()]

374         1654 
536         8117 
1665        4316 
1736        5520 
2037        4637 
            ...  
1043192    1597  
1043430    21251 
1043471    35086 
1043629    1523  
1043641    19775 
Name: income, Length: 8099, dtype: category
Categories (179865, object): [8, 14, 23, 24, ..., '9995  ', '9998', '9998  ', '99989']

In [55]:
w2_c.loc[374, 'income']

'1654 '

In [56]:
w2_c.date_of_birth

0          02/25/1979
1          03/10/1981
2          05/07/1998
3          02/23/1982
4          02/23/1982
              ...    
1043909    07/20/1990
1043910    11/21/1994
1043911    07/08/1955
1043912    07/08/1955
1043913    03/19/1964
Name: date_of_birth, Length: 1043914, dtype: category
Categories (34888, object): ['00/01/1931', '00/01/1968', '00/01/1990', '00/02/1950', ..., '98/04/1939', '98/09/1936', '99/04/1937', '99/13/1939']

In [59]:
'abc'.split('_')

['abc']

In [61]:
x, *y = 'abc'.split('_')

In [62]:
x

'abc'

In [63]:
y

[]

In [65]:
int('123')

123

In [66]:
int('123 ')

123

# Check employer addresses across shards

In [67]:
w2

,simulant_id,first_name,middle_initial,last_name,age,date_of_birth,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,...,income,employer_id,employer_name,employer_street_number,employer_street_name,employer_unit_number,employer_city,employer_state,employer_zipcode,tax_form
0,1490_588,Adriana,M,Baumert,41,02/25/1979,1029,westgate way,NaN,NaN,...,55820,325622,Westminster Arts,NaN,e federal st,NaN,birmingham,AL,35648,W2
1,1490_589,David,L,Baumert,39,03/10/1981,1029,westgate way,NaN,NaN,...,77760,1598657,Empanadas Golden Boys Painting,6628,hawks haven rd,NaN,statesville,NC,28168,W2
2,1490_590,Abigael,M,Baumert,22,05/07/1998,1029,westgate way,NaN,NaN,...,9451,849149,Alamo Community Church of Education,9712,3rd street,NaN,five forks,SC,29926,W2
3,1490_1161,Ronald,D,Wright,38,02/23/1982,2115,nort elk lane,NaN,NaN,...,12349,99503,Northwest Pizza & Boutique,3223,wellesley avenue,NaN,winter springs,FL,32571,W2
4,1490_1161,Ronald,NaN,Wright,38,02/23/1982,2115,nort elk lane,NaN,NaN,...,7196,217787,Saint Helens Nail Spa,155,7th avenue,NaN,new york,NY,11217,W2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043909,5072_1025338,Joshua,NaN,Beil,30,07/20/1990,1425,old kings hwy s,NaN,NaN,...,16370,413828,Henry E DDS,3,nantucket dr,NaN,beckley,WV,25701,W2
1043910,5072_1029528,Marcus,D,Salu Jacquot,26,11/21/1994,422,katherine claire ln,NaN,NaN,...,8159,568939,GCS University Settlement Early Childhood Educ...,833,w pike st,NaN,flint,MI,48122,W2
1043911,5072_1030814,Amelia,T,Alcamo,65,07/08/1955,408,lake ridge drive nrth,NaN,NaN,...,4964,771438,The Church of Touch Miami,722,everbreeze dr,NaN,fort myers,FL,32826,W2
1043912,5072_1030814,Amelia,T,Alcamo,65,07/08/1955,408,lake ridge drive nrth,NaN,NaN,...,10930,1203472,Green Gables Estate Bellevue Dental,10005,n crescent dr,NaN,paradise,TX,75020,W2


In [68]:
w2_c

,simulant_id,first_name,middle_initial,last_name,age,date_of_birth,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,...,income,employer_id,employer_name,employer_street_number,employer_street_name,employer_unit_number,employer_city,employer_state,employer_zipcode,tax_form
0,1490000000588,Adriana,M,Baumert,41,02/25/1979,1029,westgate way,NaN,NaN,...,55820,325622,Westminster Arts,NaN,e federal st,NaN,birmingham,AL,35648,W2
1,1490000000589,David,L,Baumert,39,03/10/1981,1029,westgate way,NaN,NaN,...,77760,1598657,Empanadas Golden Boys Painting,6628,hawks haven rd,NaN,statesville,NC,28168,W2
2,1490000000590,Abigael,M,Baumert,22,05/07/1998,1029,westgate way,NaN,NaN,...,9451,849149,Alamo Community Church of Education,9712,3rd street,NaN,five forks,SC,29926,W2
3,1490000001161,Ronald,D,Wright,38,02/23/1982,2115,nort elk lane,NaN,NaN,...,12349,99503,Northwest Pizza & Boutique,3223,wellesley avenue,NaN,winter springs,FL,32571,W2
4,1490000001161,Ronald,NaN,Wright,38,02/23/1982,2115,nort elk lane,NaN,NaN,...,7196,217787,Saint Helens Nail Spa,155,7th avenue,NaN,new york,NY,11217,W2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043909,5072001025338,Joshua,NaN,Beil,30,07/20/1990,1425,old kings hwy s,NaN,NaN,...,16370,413828,Henry E DDS,3,nantucket dr,NaN,beckley,WV,25701,W2
1043910,5072001029528,Marcus,D,Salu Jacquot,26,11/21/1994,422,katherine claire ln,NaN,NaN,...,8159,568939,GCS University Settlement Early Childhood Educ...,833,w pike st,NaN,flint,MI,48122,W2
1043911,5072001030814,Amelia,T,Alcamo,65,07/08/1955,408,lake ridge drive nrth,NaN,NaN,...,4964,771438,The Church of Touch Miami,722,everbreeze dr,NaN,fort myers,FL,32826,W2
1043912,5072001030814,Amelia,T,Alcamo,65,07/08/1955,408,lake ridge drive nrth,NaN,NaN,...,10930,1203472,Green Gables Estate Bellevue Dental,10005,n crescent dr,NaN,paradise,TX,75020,W2


In [69]:
w2_c.simulant_id.value_counts()

2721000912572    7
5046000328771    7
5114000736833    7
3304000818598    6
2965000931899    6
                ..
7137000830882    1
7137000831420    1
7137000831562    1
7137000833626    1
5072001032414    1
Name: simulant_id, Length: 757448, dtype: int64

In [70]:
w2_c.employer_id.value_counts()

1          4102
80980        43
1389791      43
1            41
596498       39
           ... 
729141        1
729139        1
729138        1
729135        1
999964        1
Name: employer_id, Length: 634812, dtype: int64

In [74]:
w2_c.employer_id.value_counts().index

CategoricalIndex([1, 80980, 1389791, '1      ', 596498, 970539, 891635, '1',
                  904393, 404051,
                  ...
                  729153, 729150, 729148, 729145, 729142, 729141, 729139,
                  729138, 729135, '999964 '],
                 categories=[1, 2, 3, 6, ..., '999615 ', '999833', '999922', '999964 '], ordered=False, dtype='category', length=634812)

In [76]:
w2.query("employer_id == 80980")

,simulant_id,first_name,middle_initial,last_name,age,date_of_birth,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,...,income,employer_id,employer_name,employer_street_number,employer_street_name,employer_unit_number,employer_city,employer_state,employer_zipcode,tax_form
2857,1490_972262,Christian,T,Strong,23,04/20/1997,3503,springside way,NaN,NaN,...,28003,80980,Clinch Vly Sch,575,selborne la,NaN,marque,TX,77039,W2
7424,5046_526026,Bettye,S,Masopust,80,04/27/1940,7440,e 163rd street,NaN,NaN,...,3950,80980,Clinch Vly Sch,32,n brown ave,NaN,la porte,TX,77039,W2
19988,5004_863151,Rebecca,S,Hoagland,25,05/04/1995,2456,w garfield st,NaN,NaN,...,13421,80980,Clinch Vly Sch,318,n sr 235,NaN,oakland,TX,77093,W2
26561,1832_252597,Jose,A,White,39,06/28/1981,1461,kimbleton ave,NaN,NaN,...,18208,80980,Clinch Vly Sch,12240,nw harris st,NaN,fort worth,TX,77026,W2
40417,6793_879099,Gino,M,Hull,24,10/14/1996,n53w 37214,orangeview ave,NaN,NaN,...,15294,80980,Clinch Vly Sch,802,klamath falls dr,NaN,conroe,TX,77032,W2
54741,4528_877154,Joseph,A,Miller,33,01/28/1987,3311,covington ln,NaN,NaN,...,64045,80980,Clinch Vly Sch,219,bagby court,NaN,hillcrest,TX,77093,W2
77667,5619_844785,David,D,Wine,69,06/20/1951,5130,s parker rd,NaN,NaN,...,54625,80980,Clinch Vly Sch,711,orchlee st,NaN,oletha,TX,77039,W2
146800,4943_396797,Josephine,K,Houska,19,10/08/2001,1710,flying a trl,NaN,NaN,...,1825,80980,Clinch Vly Sch,5053,hickory marsh ln,NaN,oak ridge,TX,77060,W2
164062,1777_629973,Elaine,L,Ashley,65,09/13/1955,3130,railroad st,NaN,NaN,...,9373,80980,Clinch Vly Sch,7143,canterbury lane,NaN,brownwood,TX,77060,W2
172911,1648_703406,Jesse,R,Whitacre,46,12/29/1974,e9424,bayview ave,NaN,NaN,...,55612,80980,Clinch Vly Sch,614,skylark drive,NaN,houston,TX,77039,W2


In [79]:
w2_c.loc[w2_c.employer_id==80980].filter(regex="simulant|employer").sort_values('simulant_id')

,simulant_id,employer_id,employer_name,employer_street_number,employer_street_name,employer_unit_number,employer_city,employer_state,employer_zipcode
584631,40000911527,80980,Clinch Vly Sch,661,cochise rd,NaN,houston,TX,77060
603205,99000157237,80980,Clinch Vly Sch,386,harmon avenue,NaN,the colony,TX,77060
991912,315000093525,80980,Clinch Vly Sch,8480,birch ln,NaN,van,TX,77039
663983,446000999098,80980,Clinch Vly Sch,1078,n sunapee loop,NaN,throckmorton,TX,77039
932433,1452000530381,80980,Clinch Vly Sch,7951,nw 302nd st,aprt 4,richardson,TX,77093
2857,1490000972262,80980,Clinch Vly Sch,575,selborne la,NaN,marque,TX,77039
172911,1648000703406,80980,Clinch Vly Sch,614,skylark drive,NaN,houston,TX,77039
164062,1777000629973,80980,Clinch Vly Sch,7143,canterbury lane,NaN,brownwood,TX,77060
26561,1832000252597,80980,Clinch Vly Sch,12240,nw harris st,NaN,fort worth,TX,77026
994557,2284000004186,80980,Clinch Vly Sch,41,duff rd,NaN,georgetown,TX,77026


In [81]:
w2_c.simulant_id // 10**datatypes.ID_PAD_WIDTH

0          1490
1          1490
2          1490
3          1490
4          1490
           ... 
1043909    5072
1043910    5072
1043911    5072
1043912    5072
1043913    5072
Name: simulant_id, Length: 1043914, dtype: int64

In [85]:
%%time
w2_c.employer_id.astype(str).str.strip().str.isdigit().all()

CPU times: user 1.16 s, sys: 3.6 ms, total: 1.17 s
Wall time: 1.16 s


False

In [86]:
%%time
eid = w2_c.employer_id.astype(str).str.strip()
eid.loc[~eid.str.isdigit()]

CPU times: user 1.04 s, sys: 19.7 ms, total: 1.06 s
Wall time: 1.06 s


58         nan
154        nan
193        nan
194        nan
202        nan
          ... 
1043333    nan
1043413    nan
1043421    nan
1043579    nan
1043903    nan
Name: employer_id, Length: 12289, dtype: object

In [87]:
%%time
w2_c.employer_id.loc[w2_c.employer_id.notna()].astype(str).str.strip().str.isdigit().all()

CPU times: user 734 ms, sys: 44 ms, total: 778 ms
Wall time: 776 ms


True